In [1]:
import json
import joblib

from pathlib import Path

import pandas as pd
import numpy as np

import lightgbm as lgb
import catboost as cb

from isic_helper import DotDict, get_folds

In [2]:
model_names = ["cb", "lgb"]
versions = ["v1", "v3"]
paths = [Path(f"/kaggle/input/isic-scd-{model_name}-{version}-train") for model_name, version in zip(model_names, versions)]

weights = [0.4, 0.6]

In [3]:
id_column = "isic_id"
target_column = "target"
group_column = "patient_id"

In [4]:
def count_features(df, col):
    tmp = df[[id_column, group_column, col]].pivot_table(
        values=id_column, 
        index=group_column, 
        columns=col, 
        aggfunc="count", 
        fill_value=0)
    feature_cols = tmp.columns.tolist()
    tmp.reset_index(inplace=True)
    tmp.index.name = None
    df = df.merge(tmp, on=group_column, how="left")
    return df, feature_cols

def mean_features(df, col, val):
    tmp = df[[id_column, group_column, col, val]].pivot_table(
        values=val, 
        index=group_column, 
        columns=col, 
        aggfunc="mean", 
        fill_value=0)
    tmp.columns = [f"{c}_{val}_mean" for c in tmp.columns.tolist()]
    feature_cols = tmp.columns.tolist()
    tmp.reset_index(inplace=True)
    tmp.index.name = None
    df = df.merge(tmp, on=group_column, how="left")
    return df, feature_cols


def stat_features(df, group_cols, value_col, stats):
    tmp = df.groupby(group_cols)[value_col].agg(stats)
    tmp.columns = [f"{value_col}_{stat}" for stat in stats]
    tmp.reset_index(inplace=True)
    df = df.merge(tmp, on=group_cols, how="left")
    df[f"{value_col}_mean_diff"] = df[value_col] - df[f"{value_col}_mean"]
    return df


def feature_engineering(df):
    new_num_cols = []
    
    df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
    new_num_cols += ["lesion_size_ratio"]
    
    df["lesion_distance"] = np.sqrt(df["tbp_lv_x"]**2 + df["tbp_lv_y"]**2 + df["tbp_lv_z"]**2)
    new_num_cols += ["lesion_distance"]
    
    df["hue_contrast"] = df["tbp_lv_H"] - df["tbp_lv_Hext"]
    df, feature_cols = mean_features(df, "anatom_site_general", "hue_contrast")
    new_num_cols += feature_cols
    
    df, feature_cols = count_features(df, "anatom_site_general")
    new_num_cols += feature_cols
    
    df["tbp_lv_A_diff"] =  df["tbp_lv_Aext"] - df["tbp_lv_A"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_A_diff", ["mean"])
    new_num_cols += ["tbp_lv_A_diff_mean_diff"]
    
    df["tbp_lv_B_diff"] =  df["tbp_lv_Bext"] - df["tbp_lv_B"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_B_diff", ["mean"])
    new_num_cols += ["tbp_lv_B_diff_mean_diff"]
    
    df["tbp_lv_L_diff"] =  df["tbp_lv_Lext"] - df["tbp_lv_L"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_L_diff", ["mean"])
    new_num_cols += ["tbp_lv_L_diff_mean_diff"]
    
    df["tbp_lv_L_std_diff"] =  df["tbp_lv_stdLExt"] - df["tbp_lv_stdL"]
    df = stat_features(df, ["patient_id", "tbp_lv_location"], "tbp_lv_L_std_diff", ["mean"])
    new_num_cols += ["tbp_lv_L_std_diff_mean_diff"]
    
    df["color_uniformity"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_radial_color_std_max"]
    df, feature_cols = mean_features(df, "anatom_site_general", "color_uniformity")
    new_num_cols += feature_cols
    
    df["radius"] = np.cos(df["tbp_lv_symm_2axis_angle"]) * np.sqrt(df["tbp_lv_x"]**2 + df["tbp_lv_y"]**2 + df["tbp_lv_z"]**2)
    new_num_cols += ["radius"]
    
    return df, new_num_cols

In [5]:
INPUT_PATH = Path("../input/isic-2024-challenge/")

train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False)
test_metadata = pd.read_csv(INPUT_PATH / "test-metadata.csv")

folds_df = get_folds()
train_metadata = train_metadata.merge(folds_df, on=["isic_id", "patient_id"], how="inner")
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

train_metadata, new_num_cols = feature_engineering(train_metadata.copy())
test_metadata, _ = feature_engineering(test_metadata.copy())

Train data size: (401059, 57)
Test data size: (3, 44)


In [6]:
def get_boosting_predictions(train, test, model_name, version, path):
    with open(path / f"{model_name}_{version}_encoder.joblib", "rb") as f:
        mixed_encoded_preprocessor = joblib.load(f)

    enc = mixed_encoded_preprocessor.fit(train)

    for col in mixed_encoded_preprocessor.feature_names_in_:
        if col not in test.columns:
            test[col] = np.nan

    X_test = enc.transform(test)

    columns_for_model = len(X_test.columns)
    print(f"Total number of columns: {columns_for_model}")

    with open(path / f"{model_name}_{version}_run_metadata.json", "r") as f:
        run_metadata = json.load(f)
        
    all_folds = np.unique(train["fold"])
    test_predictions_df = pd.DataFrame({id_column: test_metadata[id_column]})
    for fold in all_folds:
        model_filepath = path / f"models/{model_name}_{version}_fold_{fold}.txt"
        if "lgb" in model_name:
            model = lgb.Booster(model_file=model_filepath)
            test_predictions_df[f"fold_{fold}"] = model.predict(X_test, num_iteration=run_metadata["best_num_rounds"][f"fold_{fold}"])
        elif "cb" in model_name:
            model = cb.CatBoostClassifier(use_best_model=True)
            model.load_model(model_filepath)
            test_predictions_df[f"fold_{fold}"] = model.predict_proba(X_test)[:, -1]
    test_predictions_df[target_column] = test_predictions_df[[f"fold_{fold}" for fold in all_folds]].mean(axis=1)
    return test_predictions_df[[id_column, target_column]]

In [7]:
model_predict_function_topology = {
    "lgb": get_boosting_predictions,
    "cb": get_boosting_predictions
}

In [8]:
ensemble_preds = 0
previous_model_name = None
for idx, (model_name, version, path, weight) in enumerate(zip(model_names, versions, paths, weights)):
    print(f"Generating predictions for {model_name}_{version}")
    model_preds_df = model_predict_function_topology[model_name](train_metadata, test_metadata, model_name, version, path)
    if idx == 0:
        ensemble_preds_df = model_preds_df.copy()
    else:
        ensemble_preds_df = ensemble_preds_df.merge(model_preds_df, on=id_column, how="inner", suffixes=(f"_{previous_model_name}", ""))
    ensemble_preds += ensemble_preds_df[target_column].rank(pct=True).values * weight
    previous_model_name = model_name
    print("\n")
ensemble_preds_df.rename(columns={target_column: f"{target_column}_{previous_model_name}"}, inplace=True)
ensemble_preds_df[target_column] = ensemble_preds

Generating predictions for cb_v1
Total number of columns: 60


Generating predictions for lgb_v3
Total number of columns: 60




In [9]:
ensemble_preds_df.head()

,isic_id,target_cb,target_lgb,target
0,ISIC_0015657,0.000182,0.000036,0.800000
1,ISIC_0015729,0.000023,0.000028,0.333333
2,ISIC_0015740,0.000113,0.000049,0.866667


In [10]:
ensemble_preds_df[target_column].describe()

count    3.000000
mean     0.666667
std      0.290593
min      0.333333
25%      0.566667
50%      0.800000
75%      0.833333
max      0.866667
Name: target, dtype: float64

In [11]:
ensemble_preds_df[[id_column, target_column]].head()

,isic_id,target
0,ISIC_0015657,0.800000
1,ISIC_0015729,0.333333
2,ISIC_0015740,0.866667


In [12]:
ensemble_preds_df[[id_column, target_column]].to_csv("submission.csv", index=False)